In [758]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np

In [929]:
def job_rating(card):
    try:
        job_rating=card.find('span','ratingsDisplay').a.text
    except AttributeError:
        job_rating=''
    return job_rating

def job_salary(card):
    try:
        job_salary=card.find(class_='salary-snippet-container').text
    except AttributeError:
        job_salary=''
    if job_salary=='':
        try:
            job_salary=card.find(class_='metadata estimated-salary-container').text
        except AttributeError:
            job_salary=''
    return job_salary

def get_job_id(soup):
    jid_list=[]
    find_jid = re.compile(r'(jobKeysWithInfo\[([a-zA-Z0-9]{16})\])')
    script = soup.find("script", text=lambda text: text and "var jobKeysWithInfo" in text).text
    for row in script.split('\n'):
            if row.startswith('jobKeysWithInfo'):
                jid = find_jid.search(row.replace("'", ""))    
                if jid:
                    jid = find_jid.search(row.replace("'", "")).group(2)
                    jid_list.append(jid)
    return jid_list


def clean_salary(df_basic):
    resultlist=[]
    mean_list=[]
    for i in df_basic['salary']:
        # print(i)
        # x=i.replace(".", "")
        x=re.sub(",", "", i)
        result=re.findall('[0-9.]+',x)
        time_measure=re.findall('[A-Za-z]+',x)
        result=[float(r) for r in result]
        if 'K' in time_measure:
            result=[1000*r for r in result]
        else:
            result=result
        if 'year' in time_measure:
            result=result
            mean=np.mean(result)
        elif 'month' in x:
            result=[12*i for i in result]
            mean=np.mean(result)         
        else:
            result=[]
        resultlist.append(result)
        mean_list.append(mean)    
    df_basic['salary_scale']=resultlist
    df_basic['salary_mean']=mean_list
    return df_basic


In [918]:
df = pd.read_csv('merged_searched_job_html.csv')
df_basic = pd.DataFrame(columns=("job_title","company_name","location_in_detail",'salary','rating',"job_id",'title','location'))
for index,row in df.iterrows():
    soup=BeautifulSoup(row[1],'html.parser')
    script = soup.find("script", text=lambda text: text and "var jobKeysWithInfo" in text).text
    """ get list of id and list of "company_name","location",'salary','rating',"job_id" """
    id_list=get_job_id(soup)
    cards=soup.find_all('div','job_seen_beacon')
    result=[[card.h2.text,card.find("div",'heading6').contents[0].text,card.find('div','companyLocation').text,
             job_salary(card),job_rating(card)] for card in cards]
    """join two list """
    [result[i].append(id_list[i]) for i in range(len(result))]
    """Add salary """
    [result[i].append(row[2]) for i in range(len(result))]
    """Add location """
    [result[i].append(row[3]) for i in range(len(result))]
    for r in range(len(result)):
        df_basic=df_basic.append(pd.Series(result[r], index = ["job_title","company_name","location_in_detail",'salary','rating',"job_id",'title','location']), ignore_index=True)
    """return into dataframe with order"""
    df_basic=df_basic[["job_title","company_name","location_in_detail",'salary','rating',"job_id",'title','location']]
    '''clean salary '''

In [925]:
resultlist=[]
mean_list=[]
for i in df_basic['salary']:
    # print(i)
    # x=i.replace(".", "")
    x=re.sub(",", "", i)
    result=re.findall('[0-9.]+',x)
    time_measure=re.findall('[A-Za-z]+',x)
    result=[float(r) for r in result]
    if 'K' in time_measure:
        result=[1000*r for r in result]
    else:
        result=result
    if 'year' in time_measure:
        result=result
        mean=np.mean(result)
    elif 'month' in x:
        result=[12*i for i in result]
        mean=np.mean(result)         
    else:
        result=[]
        mean=''
    resultlist.append(result)
    mean_list.append(mean)
df_basic['salary_scale']=resultlist
df_basic['salary_mean']=mean_list    

In [930]:
#Set the structual of dataframe
def df_basic(raw):
    df = pd.read_csv(raw)
    df_basic = pd.DataFrame(columns=("job_title","company_name","location_in_detail",'salary','rating',"job_id",'title','location'))
    for index,row in df.iterrows():
        soup=BeautifulSoup(row[1],'html.parser')
        script = soup.find("script", text=lambda text: text and "var jobKeysWithInfo" in text).text
        """ get list of id and list of "company_name","location",'salary','rating',"job_id" """
        id_list=get_job_id(soup)
        cards=soup.find_all('div','job_seen_beacon')
        result=[[card.h2.text,card.find("div",'heading6').contents[0].text,card.find('div','companyLocation').text,
                job_salary(card),job_rating(card)] for card in cards]
        """join two list """
        [result[i].append(id_list[i]) for i in range(len(result))]
        """Add salary """
        [result[i].append(row[2]) for i in range(len(result))]
        """Add location """
        [result[i].append(row[3]) for i in range(len(result))]
        for r in range(len(result)):
            df_basic=df_basic.append(pd.Series(result[r], index = ["job_title","company_name","location_in_detail",'salary','rating',"job_id",'title','location']), ignore_index=True)
        """return into dataframe with order"""
        df_basic=df_basic[["job_title","company_name","location_in_detail",'salary','rating',"job_id",'title','location']]
        '''clean salary '''
        df_basic=clean_salary(df_basic)
        '''output result.csv'''
        df_basic.to_csv('basic-info.csv',index=False, encoding='utf-8')
    return df_basic


In [931]:
if __name__ == "__main__":
    df=df_basic('merged_searched_job_html.csv')